In [85]:
# Import necessary libraries
import os
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from matplotlib import pyplot as plt
from pathlib import Path
import random

In [86]:
# ANSI escape code for green text
GREEN = "\033[32m"
RED = "\033[31m"
RESET = "\033[0m"

# List of available directory options
available_dirs = [
    "CoronaBorealis", "Lacerta", "LeoMinor", "Lepus", "Norma", 
    "Pisces", "Sextans", "Virgo"
]

# Prompt user to choose from the list
print("Please select an input directory from the following options:")
for idx, dir_name in enumerate(available_dirs, 1):
    print(f"{idx}. {dir_name}")

print("\n") # Add a newline for better readability

# Input validation for user selection
while True:
    try:
        choice = int(input(f"Enter the number of the directory (1-{len(available_dirs)}): "))
        if 1 <= choice <= len(available_dirs):
            input_name = available_dirs[choice - 1]
            break
        else:
            print(f"{RED}Invalid choice. Please choose a number between 1 and {len(available_dirs)}!{RESET}")
    except ValueError:
        print(f"{RED}Invalid input. Please enter a number!{RESET}")

# Set the directory for input images and output folder for augmented images
input_dir = Path(f'8CD/{input_name}')
output_dir = Path(f'8CD/{input_name}_augmented')

# Check if the input directory exists and contains files
if not input_dir.exists():
    raise FileNotFoundError(f"{RED}The input directory {input_dir} does not exist.{RESET}")
if not any(input_dir.iterdir()):
    raise FileNotFoundError(f"{RED}The input directory {input_dir} is empty.{RESET}")

# Create output directory if it does not exist
output_dir.mkdir(parents=True, exist_ok=True)

# Print the directories in green
print(f"{GREEN}Valid Input directory: {input_dir}{RESET}")
print(f"{GREEN}Valid Output directory: {output_dir}{RESET}")

Please select an input directory from the following options:
1. CoronaBorealis
2. Lacerta
3. LeoMinor
4. Lepus
5. Norma
6. Pisces
7. Sextans
8. Virgo


Valid Input directory: 8CD/Virgo
Valid Output directory: 8CD/Virgo_augmented


Adjust the brightness and contrast of the input image.

    Parameters:
    - img (numpy array or PIL Image): The input image to be processed.
    - brightness (int): The percentage of brightness adjustment (-100 to 100).
    - contrast (int): The percentage of contrast adjustment (-100 to 100).

    Returns:
    - numpy array: The adjusted image as a NumPy array.

In [87]:
def apply_brightness_contrast(img, brightness=0, contrast=0):
    # Ensure the input is a valid image (either NumPy array or PIL Image)
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)  # Convert NumPy array to PIL Image if needed
    elif not isinstance(img, Image.Image):
        raise TypeError("Input image must be a NumPy array or a PIL Image - apply_brightness_contrast")

    # Ensure brightness and contrast are within reasonable bounds
    brightness = max(-100, min(brightness, 100))
    contrast = max(-100, min(contrast, 100))

    # Apply brightness and contrast adjustments
    enhancer_b = ImageEnhance.Brightness(img)
    img = enhancer_b.enhance(1 + brightness / 100.0)
    
    enhancer_c = ImageEnhance.Contrast(img)
    img = enhancer_c.enhance(1 + contrast / 100.0)

    return np.array(img)  # Return the result as a NumPy array

Add Gaussian noise to an image.

    Parameters:
    - img (numpy array): The input image to which noise will be added.
    - mean (float): The mean of the Gaussian distribution for the noise (default: 0).
    - stddev (float): The standard deviation of the Gaussian distribution for the noise (default: 25).

    Returns:
    - numpy array: The noisy image.

In [88]:
def add_noise(img, mean=0, stddev=25):
    # Ensure the input image is a NumPy array
    if not isinstance(img, np.ndarray):
        raise TypeError("Input image must be a NumPy array - add_noise")

    # Generate Gaussian noise
    noise = np.random.normal(mean, stddev, img.shape).astype(np.float32)

    # Add the noise to the image
    noisy_img = cv2.add(img.astype(np.float32), noise)

    # Clip values to ensure they stay within valid 8-bit range [0, 255]
    noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)

    return noisy_img

Apply Gaussian blur to an image.

    Parameters:
    - img (numpy array): The input image to be blurred.
    - ksize (int): The size of the Gaussian kernel (must be an odd number and greater than 0, default: 5).

    Returns:
    - numpy array: The blurred image.

In [89]:
def apply_blur(img, ksize=5):
    # Ensure ksize is a positive odd integer
    if not isinstance(ksize, int) or ksize <= 0 or ksize % 2 == 0:
        raise ValueError("Kernel size (ksize) must be a positive odd integer - apply_blur")

    # Apply Gaussian blur with the given kernel size
    blurred_img = cv2.GaussianBlur(img, (ksize, ksize), 0)

    return blurred_img

Apply skew transformation to an image using affine transformation.

    Parameters:
    - img (numpy array): The input image to be skewed.
    - skew_factor (float): The horizontal skew factor, which affects the points for skewing (default: 0.1).
    - skew_angle (float): The vertical skew factor, controlling how much the image will be distorted along the y-axis (default: 0.33).

    Returns:
    - numpy array: The skewed image.

In [90]:
def apply_skew(img, skew_factor=0.1, skew_angle=0.33):
    # Ensure the input is a valid image (NumPy array)
    if not isinstance(img, np.ndarray):
        raise TypeError("Input image must be a NumPy array.")
    
    if img.ndim != 3 or img.shape[2] not in [3, 4]:  # Checking for RGB or RGBA images
        raise ValueError("Input image must be a 3-channel (RGB) or 4-channel (RGBA) image.")

    rows, cols, _ = img.shape

    # Define source and destination points for the affine transformation
    pts1 = np.float32([[0, 0], [cols, 0], [0, rows]])
    pts2 = np.float32([
        [cols * skew_factor, rows * skew_angle],  # Skewing the top-left corner
        [cols * (1 - skew_factor), rows * (skew_angle - 0.08)],  # Skewing the top-right corner
        [cols * skew_factor, rows * (1 - skew_angle)]  # Skewing the bottom-left corner
    ])

    # Get the affine transformation matrix
    M = cv2.getAffineTransform(pts1, pts2)

    # Apply the affine transformation (skewing)
    skewed_img = cv2.warpAffine(img, M, (cols, rows))

    return skewed_img

cale (resize) an image by a given scale factor.

    Parameters:
    - img (numpy array): The input image to be scaled.
    - scale (float): The scale factor by which to resize the image (default: 1.2).

    Returns:
    - numpy array: The resized (scaled) image.

In [91]:
def apply_scaling(img, scale=1.2):
    # Ensure the input is a valid image (NumPy array)
    if not isinstance(img, np.ndarray):
        raise TypeError("Input image must be a NumPy array.")
    
    if img.ndim != 3:
        raise ValueError("Input image must be a 3-channel (RGB) or 4-channel (RGBA) image.")
    
    if scale <= 0:
        raise ValueError("Scale factor must be a positive value.")

    # Get the original dimensions of the image
    height, width = img.shape[:2]

    # Calculate new dimensions based on the scale factor
    new_width = int(width * scale)
    new_height = int(height * scale)

    # Ensure that the new dimensions are positive
    if new_width <= 0 or new_height <= 0:
        raise ValueError("Scaling factor results in invalid image dimensions.")

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height))

    return resized_img

In [92]:
def apply_rotation(img, max_angle=15):
    """Rotate the image by a random angle within the given range."""
    rows, cols, ch = img.shape
    angle = random.uniform(-max_angle, max_angle)
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated_img = cv2.warpAffine(img, M, (cols, rows))
    return rotated_img

In [93]:
def apply_flip(img, flip_type="horizontal"):
    """Flip the image either horizontally or vertically."""
    if flip_type == "horizontal":
        return cv2.flip(img, 1)  # Horizontal flip
    elif flip_type == "vertical":
        return cv2.flip(img, 0)  # Vertical flip
    else:
        raise ValueError("flip_type must be either 'horizontal' or 'vertical'.")

In [94]:
def apply_color_jitter(img, brightness=0, contrast=0, saturation=0, hue=0):
    """Randomly adjust brightness, contrast, saturation, and hue of the image."""
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Apply brightness
    hsv_img[:, :, 2] = np.clip(hsv_img[:, :, 2] + brightness, 0, 255)
    
    # Apply contrast
    hsv_img[:, :, 1] = np.clip(hsv_img[:, :, 1] + contrast, 0, 255)
    
    # Apply saturation
    hsv_img[:, :, 1] = np.clip(hsv_img[:, :, 1] + saturation, 0, 255)
    
    # Apply hue
    hsv_img[:, :, 0] = np.clip(hsv_img[:, :, 0] + hue, 0, 179)
    
    jittered_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
    return jittered_img

In [95]:
def apply_perspective_transform(img, strength=1.0):
    """Apply a perspective transformation to the image with a given strength."""
    rows, cols, _ = img.shape

    # The strength will control how much the corners of the image are shifted
    shift_x = cols * (0.1 * strength)  # Horizontal shift based on strength
    shift_y = rows * (0.1 * strength)  # Vertical shift based on strength
    
    # Define the original and transformed points
    pts1 = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])
    pts2 = np.float32([
        [shift_x, shift_y], 
        [cols - shift_x, shift_y], 
        [shift_x, rows - shift_y], 
        [cols - shift_x, rows - shift_y]
    ])
    
    # Compute the perspective transform matrix
    M = cv2.getPerspectiveTransform(pts1, pts2)
    
    # Apply the perspective warp
    warped_img = cv2.warpPerspective(img, M, (cols, rows))
    return warped_img


In [96]:
# Loop through all images in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith((".jpg", ".png")):
        img_path = os.path.join(input_dir, filename)
        
        # Read the image and check if it's loaded correctly
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Failed to load image {filename}. Skipping...")
            continue
        
        # Apply various augmentations with different values for each transformation
        augmented_images = [
            ("brightness_contrast_+30_+30", apply_brightness_contrast(img, brightness=30, contrast=30)),
            ("brightness_contrast_-30_-30", apply_brightness_contrast(img, brightness=-30, contrast=-30)),
            ("brightness_contrast_+50_+50", apply_brightness_contrast(img, brightness=50, contrast=50)),
            ("brightness_contrast_-50_-50", apply_brightness_contrast(img, brightness=-50, contrast=-50)),
            ("noise_25", add_noise(img)),
            ("noise_50", add_noise(img, mean=0, stddev=50)),
            ("blur_ksize_3", apply_blur(img, ksize=3)),
            ("blur_ksize_5", apply_blur(img, ksize=5)),
            ("blur_ksize_9", apply_blur(img, ksize=9)),
            ("skew", apply_skew(img)),
            ("skew_stronger", apply_skew(img, skew_factor=1.5)),  # Adjust skew factor for stronger distortion
            ("scaling_1.1", apply_scaling(img, scale=1.1)),
            ("scaling_0.9", apply_scaling(img, scale=0.9)),
            ("scaling_1.2", apply_scaling(img, scale=1.2)),
            ("scaling_0.8", apply_scaling(img, scale=0.8)),
            ("rotation_15", apply_rotation(img, max_angle=15)),
            ("rotation_30", apply_rotation(img, max_angle=30)),
            ("rotation_45", apply_rotation(img, max_angle=45)),
            ("flip_horizontal", apply_flip(img, flip_type="horizontal")),
            ("flip_vertical", apply_flip(img, flip_type="vertical")),
            ("color_jitter_10_10_10_5", apply_color_jitter(img, brightness=10, contrast=10, saturation=10, hue=5)),
            ("color_jitter_30_30_30_15", apply_color_jitter(img, brightness=30, contrast=30, saturation=30, hue=15)),
            ("perspective_transform", apply_perspective_transform(img)),
            ("perspective_transform_strong", apply_perspective_transform(img, strength=1.5))  # Adjust strength of perspective warp
        ]

        # Additional transformations to add:
        augmented_images.extend([
            ("rotation_60", apply_rotation(img, max_angle=60)),
            ("scaling_1.3", apply_scaling(img, scale=1.3)),
            ("scaling_0.7", apply_scaling(img, scale=0.7)),
            ("color_jitter_40_40_40_20", apply_color_jitter(img, brightness=40, contrast=40, saturation=40, hue=20)),
            ("color_jitter_60_60_60_30", apply_color_jitter(img, brightness=60, contrast=60, saturation=60, hue=30)),
            ("skew_2", apply_skew(img, skew_factor=2)),  # Even stronger skew
            ("blur_ksize_11", apply_blur(img, ksize=11)),
            ("perspective_transform_random", apply_perspective_transform(img, strength=random.uniform(0.5, 2))),
            ("noise_75", add_noise(img, mean=0, stddev=65)),
            ("rotation_random", apply_rotation(img, max_angle=random.randint(10, 60)))
        ])


        # Save augmented images with descriptive names
        for aug_name, aug_img in augmented_images:
            aug_img_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_{aug_name}.png")
            cv2.imwrite(aug_img_path, aug_img)

print("Data augmentation completed.")

Data augmentation completed.
